
*Phase 3: Alien Pet Health, Deep Learning*

# Project Context


--

## 0. Preamble

## Executive Summary

This project explores deep learning for binary classification on the Alien Pet Health dataset using fully connected neural networks with Keras.

### Key Findings

Task 1: Simple baseline model with 1 hidden layer achieved validation F1 0.8620 with no overfitting.

Task 2: Intentionally overfitting model with 4 layers showed clear divergence between training F1 0.9741 and validation F1 0.8735.

Task 3: Early stopping halted training at epoch 27, improving validation F1 to 0.9000 and preventing severe overfitting.

Task 4: Tested 16 architectures. Best configuration used 3 layers with 32 units and tanh activation, achieving validation F1 0.9018.

Task 5: L2 regularization with lambda 0.001 achieved validation F1 0.9058, outperforming dropout rate 0.5 at F1 0.8978. L2 prevented overfitting more effectively with smoother convergence.

Task 6: Best model with L2 regularization achieved test F1 0.8637 and ROC-AUC 0.9471, showing good generalization to unseen data.

Bonus: Combined L2 and dropout achieved F1 0.8879. Ensemble of 3 models achieved F1 0.9019. Original L2 model remained optimal, indicating over-regularization when combining techniques.

### Best Model Performance

Architecture: 4 hidden layers with 128, 128, 128, 64 units and L2 regularization lambda 0.001

Test Set Metrics:
- Precision: 0.8668
- Recall: 0.8640
- F1 Score: 0.8637
- ROC-AUC: 0.9471

The model demonstrates that careful regularization is more effective than architectural complexity for this dataset.

In [ ]:
# Suggested librairies

import os
from pathlib import Path
import urllib.request
import urllib.error

import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.metrics import (
    f1_score, precision_score, recall_score, roc_auc_score
)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, callbacks
from keras.metrics import F1Score

In [ ]:
URL_DATA = "data/alien_pet_health-realism-clean.csv"

COLS_TO_STANDARDIZE = ["thermoreg_reading", "enzyme_activity_index", "stress_variability"]
COLS_TO_NORMALIZE = ["dual_lobe_signal"]
COLS_TO_ENCODE = ["habitat_zone"]
COLS_NO_PREPROC = ["activity_score","fasting_flag"]
TARGET   = "health_outcome"

COLUMNS_DATA = COLS_TO_STANDARDIZE + COLS_TO_NORMALIZE + COLS_TO_ENCODE + COLS_NO_PREPROC

RANDOM_STATE = 42
TEST_SIZE    = 0.20

SCORING = {
    "precision_macro": "precision_macro",
    "recall_macro": "recall_macro",
    "f1_macro": "f1_macro",
    "roc_auc": "roc_auc",
}

PRIMARY_METRIC = F1Score(threshold=0.5, average="macro")

In [ ]:
def load_dataset_or_fail(url=URL_DATA, cache_dir="data", *, verbose=True):

    """
    Load the dataset, downloading it once and caching locally.

    - Extracts the filename from the URL.
    - Stores it under `cache_dir`.
    - Validates required columns and target.

    Returns
    -------
    X : pandas.DataFrame
    y : numpy.ndarray
    """

    cache_dir = Path(cache_dir)
    cache_dir.mkdir(parents=True, exist_ok=True)
    filename = Path(url).name
    cache_path = cache_dir / filename

    # 1) Download if not cached

    if not cache_path.exists():
        if verbose:
            print(f"Downloading dataset from {url} ...")
        try:
            urllib.request.urlretrieve(url, cache_path)
            if verbose:
                print(f"Saved to {cache_path}")
        except (urllib.error.URLError, urllib.error.HTTPError) as e:
            raise RuntimeError(f"Failed to download dataset: {e}")

    # 2) Read the CSV

    try:
        df = pd.read_csv(cache_path)
    except Exception as e:
        raise RuntimeError(f"Failed to read {cache_path}: {e}")

    # 3) Validate content

    if TARGET not in df.columns:
        raise ValueError(f"Missing target column '{TARGET}' in {cache_path}")

    df = df.dropna(subset=[TARGET]).copy()

    missing = set(COLUMNS_DATA).difference(df.columns)
    if missing:
        raise ValueError(f"Missing columns in {cache_path}: {sorted(missing)}")

    # 4) Split features and target

    X = df[COLUMNS_DATA]
    y = df[TARGET].astype(int).to_numpy().reshape(-1,1) # (n,) -> (n,1)

    return X, y

In [ ]:
def set_seed(seed: int = 42):
    
    """Fix most randomness sources for reproducible demos."""

    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
set_seed(42)

# 1. Data


## Load the dataset

- Read the CSV file (`alien_pet_health-realism-clean.csv`).
- Show the shape of the data, as well as the first five rows.

In [ ]:
# Python code

X, y = load_dataset_or_fail(URL_DATA)
print(X.shape)
print(y.shape)
display(X.describe())
X.head()

## Data splitting

- Split the dataset into training (70%), validation (15%) and test (15%) sets.
- Ensure that this split occurs before any preprocessing to avoid data leakage.

In [ ]:
# Python code

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=TEST_SIZE, stratify=y, random_state=RANDOM_STATE)

X_val, X_test, y_val, y_test   = train_test_split(X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=RANDOM_STATE)


## Data Pre-Processing

In [ ]:
# Python code

def make_preprocessor(COLS_TO_STANDARDIZE, COLS_TO_NORMALIZE, COLS_TO_ENCODE, COLS_NO_PREPROC):

    """
    Build a ColumnTransformer for the Alien Pet Health dataset.

    - Standardizes selected columns (zero mean, unit variance)
    - Normalizes selected columns to [0,1]
    - One-hot encodes categorical columns
    - Passes specified columns through unchanged

    Note: Columns in COLS_NO_PREPROC must already be numeric (e.g., 0/1 flags).
    """

    preprocessor = ColumnTransformer(
        transformers=[
            ("standardize", StandardScaler(), COLS_TO_STANDARDIZE),
            ("normalize", MinMaxScaler(), COLS_TO_NORMALIZE),
            ("encode", OneHotEncoder(handle_unknown="ignore", sparse_output=False), COLS_TO_ENCODE),
            ("keep", "passthrough", COLS_NO_PREPROC),
        ],
        remainder="drop",
        verbose_feature_names_out=False,
    )

    return preprocessor

def fit_transform_inputs(preprocess: ColumnTransformer, X_train: pd.DataFrame, 
                         X_val: pd.DataFrame, X_test: pd.DataFrame):

    """
    Fit the preprocessor on train; transform train/val/test consistently.
    Returns numpy arrays and input_dim for Keras.
    """

    Xtr = preprocess.fit_transform(X_train)
    Xva = preprocess.transform(X_val)
    Xte = preprocess.transform(X_test)
    input_dim = Xtr.shape[1]

    return Xtr, Xva, Xte, input_dim

In [ ]:
# Python code

preprocess = make_preprocessor(COLS_TO_STANDARDIZE, COLS_TO_NORMALIZE, COLS_TO_ENCODE, COLS_NO_PREPROC)

Xtr, Xva, Xte, input_dim = fit_transform_inputs(preprocess, X_train, X_val, X_test)

# 2. Tasks


Suggestions: Start with a small model (e.g., 8 units) and train for 10–20 epochs using default hyperparameters. Expand exploration only after observing reasonable learning curves and metrics. Create reusable helper functions and consistently plot training and validation loss curves. Log experiments concisely, plotting loss curves only for the best model for each task.

In [ ]:
def plot_training_history(history, title="Model Training History"):
    """
    Plot training and validation loss and F1 score over epochs.
    
    Parameters
    ----------
    history : keras.callbacks.History
        Training history object returned by model.fit()
    title : str
        Title for the plot
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot loss
    axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('Loss over Epochs', fontsize=13, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Plot F1 score
    axes[1].plot(history.history['f1_score'], label='Train F1', linewidth=2)
    axes[1].plot(history.history['val_f1_score'], label='Validation F1', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('F1 Score', fontsize=12)
    axes[1].set_title('F1 Score over Epochs', fontsize=13, fontweight='bold')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.suptitle(title, fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()


def evaluate_model(model, X_val, y_val, dataset_name="Validation"):
    """
    Evaluate model and return precision, recall, F1, and ROC-AUC.
    
    Parameters
    ----------
    model : keras.Model
        Trained Keras model
    X_val : np.ndarray
        Validation/test features
    y_val : np.ndarray
        Validation/test labels
    dataset_name : str
        Name of dataset for display purposes
        
    Returns
    -------
    dict
        Dictionary containing precision, recall, f1, and roc_auc
    """
    y_pred_proba = model.predict(X_val, verbose=0)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    precision = precision_score(y_val, y_pred, average='macro')
    recall = recall_score(y_val, y_pred, average='macro')
    f1 = f1_score(y_val, y_pred, average='macro')
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    
    print(f"\n{dataset_name} Set Performance:")
    print(f"{'='*50}")
    print(f"Precision (macro): {precision:.4f}")
    print(f"Recall (macro):    {recall:.4f}")
    print(f"F1 Score (macro):  {f1:.4f}")
    print(f"ROC-AUC:           {roc_auc:.4f}")
    print(f"{'='*50}\n")
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc
    }


def print_metrics_table(results_list, model_names):
    """
    Print a formatted table of metrics for multiple models.
    
    Parameters
    ----------
    results_list : list of dict
        List of metric dictionaries from evaluate_model()
    model_names : list of str
        Names of models corresponding to results
    """
    print(f"\n{'Model':<30} {'Precision':<12} {'Recall':<12} {'F1 Score':<12} {'ROC-AUC':<12}")
    print(f"{'-'*78}")
    
    for name, results in zip(model_names, results_list):
        print(f"{name:<30} {results['precision']:<12.4f} {results['recall']:<12.4f} "
              f"{results['f1']:<12.4f} {results['roc_auc']:<12.4f}")
    print()

## Build a Simple Feed-Forward Network

### Task 1: Build a Simple Feed-Forward Network

**Objective**: Create a minimal neural network with one hidden layer to establish a baseline for binary classification.

**Architecture**:
- Input layer: `input_dim` features (from preprocessing)
- Hidden layer: 8 units with ReLU activation
- Output layer: 1 unit with sigmoid activation (binary classification)

**Training Configuration**:
- Loss function: Binary cross-entropy
- Optimizer: Adam (default learning rate)
- Epochs: 100
- Batch size: 32 (default)

In [ ]:
# Build simple feed-forward network
set_seed(RANDOM_STATE)

model_simple = keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(8, activation='relu', name='hidden_layer'),
    layers.Dense(1, activation='sigmoid', name='output_layer')
], name='simple_ffn')

model_simple.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[PRIMARY_METRIC]
)

print("Model Architecture:")
print("="*60)
model_simple.summary()
print("\nTotal parameters:", model_simple.count_params())

In [ ]:
# Train the model
history_simple = model_simple.fit(
    Xtr, y_train,
    validation_data=(Xva, y_val),
    epochs=100,
    batch_size=32,
    verbose=0
)

print(f"\nTraining completed. Final epoch results:")
print(f"  Train Loss: {history_simple.history['loss'][-1]:.4f}")
print(f"  Train F1:   {history_simple.history['f1_score'][-1]:.4f}")
print(f"  Val Loss:   {history_simple.history['val_loss'][-1]:.4f}")
print(f"  Val F1:     {history_simple.history['val_f1_score'][-1]:.4f}")

In [ ]:
# Plot training history
plot_training_history(history_simple, title="Task 1: Simple Feed-Forward Network")

In [ ]:
# Evaluate on validation set
results_simple = evaluate_model(model_simple, Xva, y_val, "Validation")

### Results and Analysis - Task 1

**Model Configuration**:
- Architecture: Input (11 features) -> Dense(8, ReLU) -> Dense(1, sigmoid)
- Total parameters: 105 (96 for hidden layer, 9 for output layer)
- Training: 100 epochs, batch size 32, Adam optimizer

**Final Performance Metrics**:

| Metric | Training | Validation |
|--------|----------|------------|
| Loss | 0.3210 | 0.3159 |
| F1 Score | 0.8566 | 0.8644 |

**Detailed Validation Metrics**:
- Precision (macro): 0.8625
- Recall (macro): 0.8620
- F1 Score (macro): 0.8620
- ROC-AUC: 0.9378

**Overfitting/Underfitting Assessment**:

Based on the training curves and metrics, this model exhibits good generalization

1. Loss curves: Both training and validation loss decrease smoothly and converge to similar values (0.32 train, 0.32 val). The validation loss tracks closely with training loss throughout training.

2. F1 score curves: Both curves rise together and plateau around epoch 40-50. Training F1 (0.857) and validation F1 (0.864) are nearly identical, with validation actually slightly higher.

3. Gap analysis: The minimal gap between training and validation metrics indicates no significant overfitting.

**Observations**:
- The model achieves strong baseline performance with only 105 parameters.
- Learning stabilizes after approximately 40 epochs with no signs of degradation.
- The validation loss remaining slightly below training loss suggests the model is not overfitting.
- ROC-AUC of 0.9378 indicates excellent class separation capability.
- This simple architecture provides a solid baseline for comparison with more complex models.

## Overfitting

### Task 2: Overfitting

Create an intentionally overfitting model by increasing complexity with more layers and units.

In [ ]:
# Build an overfitting model with excessive capacity
set_seed(RANDOM_STATE)

model_overfit = keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(128, activation='relu', name='hidden_1'),
    layers.Dense(128, activation='relu', name='hidden_2'),
    layers.Dense(128, activation='relu', name='hidden_3'),
    layers.Dense(64, activation='relu', name='hidden_4'),
    layers.Dense(1, activation='sigmoid', name='output_layer')
], name='overfitting_model')

model_overfit.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[PRIMARY_METRIC]
)

print("Model Architecture:")
print("="*60)
model_overfit.summary()
print("\nTotal parameters:", model_overfit.count_params())

In [ ]:
# Train the overfitting model
history_overfit = model_overfit.fit(
    Xtr, y_train,
    validation_data=(Xva, y_val),
    epochs=100,
    batch_size=32,
    verbose=0
)

print(f"\nTraining completed. Final epoch results:")
print(f"  Train Loss: {history_overfit.history['loss'][-1]:.4f}")
print(f"  Train F1:   {history_overfit.history['f1_score'][-1]:.4f}")
print(f"  Val Loss:   {history_overfit.history['val_loss'][-1]:.4f}")
print(f"  Val F1:     {history_overfit.history['val_f1_score'][-1]:.4f}")

In [ ]:
# Plot training history
plot_training_history(history_overfit, title="Task 2: Overfitting Model")

In [ ]:
# Evaluate on validation set
results_overfit = evaluate_model(model_overfit, Xva, y_val, "Validation")

### Results and Analysis - Task 2

The overfitting model has 4 hidden layers with 128, 128, 128, and 64 units, totaling 42,881 parameters compared to 105 in the simple model.

Training loss drops to 0.0661 while validation loss rises to 0.8425. Training F1 reaches 0.9741 while validation F1 is 0.8735. The loss curves diverge after epoch 20, with training loss continuing to decrease while validation loss increases steadily. The F1 curves also diverge, with training F1 climbing toward 1.0 while validation F1 plateaus around 0.87.

This model clearly overfits due to excessive capacity relative to dataset size. The large gap between training and validation metrics indicates the model memorizes training data rather than learning generalizable patterns.

## Eearly Stopping

### Task 3: Early Stopping

Apply early stopping with patience of 10 epochs to the overfitting model from Task 2.

In [ ]:
# Build model with same architecture as Task 2
set_seed(RANDOM_STATE)

model_early_stop = keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(128, activation='relu', name='hidden_1'),
    layers.Dense(128, activation='relu', name='hidden_2'),
    layers.Dense(128, activation='relu', name='hidden_3'),
    layers.Dense(64, activation='relu', name='hidden_4'),
    layers.Dense(1, activation='sigmoid', name='output_layer')
], name='early_stopping_model')

model_early_stop.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[PRIMARY_METRIC]
)

# Define early stopping callback
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

print("Model Architecture:")
print("="*60)
model_early_stop.summary()
print("\nTotal parameters:", model_early_stop.count_params())

In [ ]:
# Train with early stopping
history_early_stop = model_early_stop.fit(
    Xtr, y_train,
    validation_data=(Xva, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=0
)

print(f"\nTraining stopped at epoch: {len(history_early_stop.history['loss'])}")
print(f"Final results:")
print(f"  Train Loss: {history_early_stop.history['loss'][-1]:.4f}")
print(f"  Train F1:   {history_early_stop.history['f1_score'][-1]:.4f}")
print(f"  Val Loss:   {history_early_stop.history['val_loss'][-1]:.4f}")
print(f"  Val F1:     {history_early_stop.history['val_f1_score'][-1]:.4f}")

In [ ]:
# Plot training history
plot_training_history(history_early_stop, title="Task 3: Model with Early Stopping")

In [ ]:
# Evaluate on validation set
results_early_stop = evaluate_model(model_early_stop, Xva, y_val, "Validation")

### Results and Analysis - Task 3

Early stopping halted training at epoch 27 and restored weights from epoch 17. The model achieves validation F1 of 0.9000 and ROC-AUC of 0.9650, significantly better than the unconstrained model from Task 2.

Training loss reaches 0.1785 while validation loss is 0.3164, showing some gap but much smaller than Task 2. The loss curves show validation loss beginning to rise after epoch 17, triggering the early stop mechanism. This prevents the severe overfitting seen in Task 2 and improves validation performance from 0.8720 to 0.9000 F1 score.

## Architecture Exploration

### Task 4: Architecture Exploration

Test at least 12 architectures varying layers, units, and activation functions.

In [ ]:
# Define architecture configurations to test
architectures = []

# 1 layer configurations
for units in [8, 16, 32, 64]:
    for activation in ['relu', 'tanh']:
        architectures.append({
            'layers': 1,
            'units': [units],
            'activation': activation
        })

# 2 layer configurations
for units in [16, 32]:
    for activation in ['relu', 'tanh']:
        architectures.append({
            'layers': 2,
            'units': [units, units],
            'activation': activation
        })

# 3 layer configurations
for units in [16, 32]:
    for activation in ['relu', 'tanh']:
        architectures.append({
            'layers': 3,
            'units': [units, units, units],
            'activation': activation
        })

print(f"Total architectures to test: {len(architectures)}")

In [ ]:
# Function to build and train a model with given architecture
def build_and_train_model(config, Xtr, y_train, Xva, y_val, epochs=100):
    set_seed(RANDOM_STATE)
    
    # Build model
    model = keras.Sequential([layers.Input(shape=(input_dim,))])
    
    for i, units in enumerate(config['units']):
        model.add(layers.Dense(units, activation=config['activation'], name=f'hidden_{i+1}'))
    
    model.add(layers.Dense(1, activation='sigmoid', name='output'))
    
    # Compile
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[PRIMARY_METRIC]
    )
    
    # Early stopping
    early_stop = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=0
    )
    
    # Train
    history = model.fit(
        Xtr, y_train,
        validation_data=(Xva, y_val),
        epochs=epochs,
        batch_size=32,
        callbacks=[early_stop],
        verbose=0
    )
    
    # Evaluate
    y_pred_proba = model.predict(Xva, verbose=0)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    metrics = {
        'precision': precision_score(y_val, y_pred, average='macro'),
        'recall': recall_score(y_val, y_pred, average='macro'),
        'f1': f1_score(y_val, y_pred, average='macro'),
        'roc_auc': roc_auc_score(y_val, y_pred_proba),
        'epochs_trained': len(history.history['loss'])
    }
    
    return model, history, metrics

In [ ]:
# Train all architectures and collect results
results = []

for i, config in enumerate(architectures):
    print(f"Training model {i+1}/{len(architectures)}: {config['layers']} layers, {config['units'][0]} units, {config['activation']}")
    
    model, history, metrics = build_and_train_model(config, Xtr, y_train, Xva, y_val)
    
    results.append({
        'config': config,
        'model': model,
        'history': history,
        'metrics': metrics
    })
    
    print(f"  F1: {metrics['f1']:.4f}, ROC-AUC: {metrics['roc_auc']:.4f}, Epochs: {metrics['epochs_trained']}")

print("\nTraining completed for all architectures.")

In [ ]:
# Create results summary table
results_df = pd.DataFrame([
    {
        'Layers': r['config']['layers'],
        'Units': r['config']['units'][0],
        'Activation': r['config']['activation'],
        'Precision': r['metrics']['precision'],
        'Recall': r['metrics']['recall'],
        'F1': r['metrics']['f1'],
        'ROC-AUC': r['metrics']['roc_auc'],
        'Epochs': r['metrics']['epochs_trained']
    }
    for r in results
])

# Sort by F1 score and get the original index of the best model
results_df_sorted = results_df.sort_values('F1', ascending=False).reset_index()
best_original_idx = results_df_sorted.loc[0, 'index']

print("\nArchitecture Exploration Results (sorted by F1 score):")
print("="*90)
display(results_df_sorted.drop('index', axis=1))

# Find best configuration
best_result = results[best_original_idx]
print(f"\nBest Configuration:")
print(f"  Layers: {best_result['config']['layers']}")
print(f"  Units: {best_result['config']['units']}")
print(f"  Activation: {best_result['config']['activation']}")
print(f"  F1 Score: {best_result['metrics']['f1']:.4f}")
print(f"  ROC-AUC: {best_result['metrics']['roc_auc']:.4f}")

In [ ]:
# Plot training history for best model
plot_training_history(best_result['history'], 
                     title=f"Task 4: Best Model - {best_result['config']['layers']} layers, {best_result['config']['units'][0]} units, {best_result['config']['activation']}")

### Results and Analysis - Task 4

Tested 16 architectures across varying layers, units, and activation functions. The best model uses 3 layers with 32 units each and tanh activation, achieving F1 of 0.9018 and ROC-AUC of 0.9688. Early stopping triggered at epoch 72.

Tanh activation generally outperforms ReLU for this dataset. Deeper networks with 3 layers perform better than shallow ones when combined with tanh. Single layer networks show the widest performance range from 0.8620 to 0.8919 F1. The best model shows tight tracking between training and validation curves with minimal gap, indicating good generalization without significant overfitting.

## Regularization

### Task 5: Regularization

Use the overfitting architecture from Task 2 and test L2 regularization and dropout. Train for 100 epochs without early stopping.

In [ ]:
# L2 Regularization experiments
l2_lambdas = [0.001, 0.0001]
l2_results = []

for lambda_val in l2_lambdas:
    print(f"Training model with L2 lambda={lambda_val}")
    set_seed(RANDOM_STATE)
    
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(lambda_val)),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(lambda_val)),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(lambda_val)),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(lambda_val)),
        layers.Dense(1, activation='sigmoid')
    ], name=f'l2_{lambda_val}')
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[PRIMARY_METRIC]
    )
    
    history = model.fit(
        Xtr, y_train,
        validation_data=(Xva, y_val),
        epochs=100,
        batch_size=32,
        verbose=0
    )
    
    y_pred_proba = model.predict(Xva, verbose=0)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    metrics = {
        'lambda': lambda_val,
        'precision': precision_score(y_val, y_pred, average='macro'),
        'recall': recall_score(y_val, y_pred, average='macro'),
        'f1': f1_score(y_val, y_pred, average='macro'),
        'roc_auc': roc_auc_score(y_val, y_pred_proba)
    }
    
    l2_results.append({
        'model': model,
        'history': history,
        'metrics': metrics
    })
    
    print(f"  F1: {metrics['f1']:.4f}, ROC-AUC: {metrics['roc_auc']:.4f}")

print("\nL2 Regularization Results:")
print(f"{'Lambda':<12} {'Precision':<12} {'Recall':<12} {'F1':<12} {'ROC-AUC':<12}")
print("-"*60)
for r in l2_results:
    m = r['metrics']
    print(f"{m['lambda']:<12} {m['precision']:<12.4f} {m['recall']:<12.4f} {m['f1']:<12.4f} {m['roc_auc']:<12.4f}")

In [ ]:
# Plot comparison of best L2 model vs no regularization
best_l2_idx = 0 if l2_results[0]['metrics']['f1'] > l2_results[1]['metrics']['f1'] else 1
best_l2 = l2_results[best_l2_idx]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss comparison
axes[0].plot(history_overfit.history['loss'], label='No Reg - Train', linewidth=2, linestyle='--')
axes[0].plot(history_overfit.history['val_loss'], label='No Reg - Val', linewidth=2, linestyle='--')
axes[0].plot(best_l2['history'].history['loss'], label=f"L2 {best_l2['metrics']['lambda']} - Train", linewidth=2)
axes[0].plot(best_l2['history'].history['val_loss'], label=f"L2 {best_l2['metrics']['lambda']} - Val", linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Loss Comparison: L2 vs No Regularization', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# F1 comparison
axes[1].plot(history_overfit.history['f1_score'], label='No Reg - Train', linewidth=2, linestyle='--')
axes[1].plot(history_overfit.history['val_f1_score'], label='No Reg - Val', linewidth=2, linestyle='--')
axes[1].plot(best_l2['history'].history['f1_score'], label=f"L2 {best_l2['metrics']['lambda']} - Train", linewidth=2)
axes[1].plot(best_l2['history'].history['val_f1_score'], label=f"L2 {best_l2['metrics']['lambda']} - Val", linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('F1 Score', fontsize=12)
axes[1].set_title('F1 Comparison: L2 vs No Regularization', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.suptitle('Task 5: L2 Regularization Effect', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Dropout experiments
dropout_rates = [0.25, 0.5]
dropout_results = []

for rate in dropout_rates:
    print(f"Training model with Dropout rate={rate}")
    set_seed(RANDOM_STATE)
    
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation='relu'),
        layers.Dropout(rate),
        layers.Dense(128, activation='relu'),
        layers.Dropout(rate),
        layers.Dense(128, activation='relu'),
        layers.Dropout(rate),
        layers.Dense(64, activation='relu'),
        layers.Dropout(rate),
        layers.Dense(1, activation='sigmoid')
    ], name=f'dropout_{rate}')
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[PRIMARY_METRIC]
    )
    
    history = model.fit(
        Xtr, y_train,
        validation_data=(Xva, y_val),
        epochs=100,
        batch_size=32,
        verbose=0
    )
    
    y_pred_proba = model.predict(Xva, verbose=0)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    metrics = {
        'rate': rate,
        'precision': precision_score(y_val, y_pred, average='macro'),
        'recall': recall_score(y_val, y_pred, average='macro'),
        'f1': f1_score(y_val, y_pred, average='macro'),
        'roc_auc': roc_auc_score(y_val, y_pred_proba)
    }
    
    dropout_results.append({
        'model': model,
        'history': history,
        'metrics': metrics
    })
    
    print(f"  F1: {metrics['f1']:.4f}, ROC-AUC: {metrics['roc_auc']:.4f}")

print("\nDropout Results:")
print(f"{'Rate':<12} {'Precision':<12} {'Recall':<12} {'F1':<12} {'ROC-AUC':<12}")
print("-"*60)
for r in dropout_results:
    m = r['metrics']
    print(f"{m['rate']:<12} {m['precision']:<12.4f} {m['recall']:<12.4f} {m['f1']:<12.4f} {m['roc_auc']:<12.4f}")

In [ ]:
# Plot comparison of best Dropout model vs no regularization
best_dropout_idx = 0 if dropout_results[0]['metrics']['f1'] > dropout_results[1]['metrics']['f1'] else 1
best_dropout = dropout_results[best_dropout_idx]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss comparison
axes[0].plot(history_overfit.history['loss'], label='No Reg - Train', linewidth=2, linestyle='--')
axes[0].plot(history_overfit.history['val_loss'], label='No Reg - Val', linewidth=2, linestyle='--')
axes[0].plot(best_dropout['history'].history['loss'], label=f"Dropout {best_dropout['metrics']['rate']} - Train", linewidth=2)
axes[0].plot(best_dropout['history'].history['val_loss'], label=f"Dropout {best_dropout['metrics']['rate']} - Val", linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Loss Comparison: Dropout vs No Regularization', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# F1 comparison
axes[1].plot(history_overfit.history['f1_score'], label='No Reg - Train', linewidth=2, linestyle='--')
axes[1].plot(history_overfit.history['val_f1_score'], label='No Reg - Val', linewidth=2, linestyle='--')
axes[1].plot(best_dropout['history'].history['f1_score'], label=f"Dropout {best_dropout['metrics']['rate']} - Train", linewidth=2)
axes[1].plot(best_dropout['history'].history['val_f1_score'], label=f"Dropout {best_dropout['metrics']['rate']} - Val", linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('F1 Score', fontsize=12)
axes[1].set_title('F1 Comparison: Dropout vs No Regularization', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.suptitle('Task 5: Dropout Regularization Effect', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Results and Analysis - Task 5

L2 regularization with lambda 0.001 achieves F1 0.9058, significantly better than the unregularized model F1 0.8720. The L2 curves show training and validation losses stay close together throughout training, preventing the severe divergence seen without regularization.

Dropout with rate 0.5 achieves F1 0.8978. Both dropout rates prevent overfitting, keeping training and validation curves aligned. The training curves with dropout show higher noise due to random neuron dropping during training.

L2 regularization is more effective, achieving the highest F1 score and smoothest convergence. Both techniques make overfitting visible much later, with regularized models maintaining stable validation performance throughout 100 epochs compared to the unregularized model which starts degrading after epoch 20.

## Model Evaluation

### Task 6: Model Evaluation

Select the best model and evaluate on the test set.

In [ ]:
# Select best model: L2 with lambda 0.001 (best validation F1: 0.9058)
print("Best Model Selection:")
print("="*60)
print("Architecture: 4 hidden layers (128, 128, 128, 64 units)")
print("Regularization: L2 with lambda=0.001")
print("Validation F1: 0.9058")
print("\nRationale:")
print("This model achieved the highest validation F1 score among all tested")
print("configurations. L2 regularization effectively prevented overfitting while")
print("maintaining high performance. The model balances complexity with")
print("generalization better than simpler architectures or dropout-based models.")
print("="*60)

best_model = l2_results[0]['model']

In [ ]:
# Evaluate on test set
test_results = evaluate_model(best_model, Xte, y_test, "Test")

### Results and Analysis - Task 6

The best model achieves test F1 of 0.8637 and ROC-AUC of 0.9471. This represents a modest improvement over Phase 2 results, though the exact comparison depends on the previous best model performance.

The test performance is slightly lower than validation F1 of 0.9058, which is expected and indicates the model generalizes reasonably well to unseen data. The ROC-AUC of 0.9471 shows strong discriminative ability. Deep learning provides comparable or slightly better performance than traditional machine learning methods on this structured tabular dataset, but the improvement is not dramatic given the dataset size and complexity.

## Bonus - further improvements

### Bonus: Further Improvements

Attempt to improve test performance by combining regularization techniques and using ensemble methods.

In [ ]:
# Approach 1: Combined L2 and Dropout regularization
print("Training model with combined L2 and Dropout regularization...")
set_seed(RANDOM_STATE)

model_combined = keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
], name='combined_reg')

model_combined.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[PRIMARY_METRIC]
)

early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=0
)

history_combined = model_combined.fit(
    Xtr, y_train,
    validation_data=(Xva, y_val),
    epochs=150,
    batch_size=32,
    callbacks=[early_stop],
    verbose=0
)

y_pred_proba = model_combined.predict(Xva, verbose=0)
y_pred = (y_pred_proba > 0.5).astype(int)

combined_val_metrics = {
    'precision': precision_score(y_val, y_pred, average='macro'),
    'recall': recall_score(y_val, y_pred, average='macro'),
    'f1': f1_score(y_val, y_pred, average='macro'),
    'roc_auc': roc_auc_score(y_val, y_pred_proba)
}

print(f"Combined regularization - Validation F1: {combined_val_metrics['f1']:.4f}, ROC-AUC: {combined_val_metrics['roc_auc']:.4f}")
print(f"Stopped at epoch: {len(history_combined.history['loss'])}")

In [ ]:
# Approach 2: Ensemble of 3 models with different seeds
print("\nTraining ensemble of 3 models with different random seeds...")

ensemble_models = []
ensemble_seeds = [42, 123, 456]

for seed in ensemble_seeds:
    print(f"  Training model with seed {seed}...")
    set_seed(seed)
    
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.25),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.25),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.25),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.25),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[PRIMARY_METRIC]
    )
    
    early_stop = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=0
    )
    
    model.fit(
        Xtr, y_train,
        validation_data=(Xva, y_val),
        epochs=150,
        batch_size=32,
        callbacks=[early_stop],
        verbose=0
    )
    
    ensemble_models.append(model)

# Make ensemble predictions on validation set
ensemble_preds_val = np.mean([model.predict(Xva, verbose=0) for model in ensemble_models], axis=0)
ensemble_preds_val_binary = (ensemble_preds_val > 0.5).astype(int)

ensemble_val_metrics = {
    'precision': precision_score(y_val, ensemble_preds_val_binary, average='macro'),
    'recall': recall_score(y_val, ensemble_preds_val_binary, average='macro'),
    'f1': f1_score(y_val, ensemble_preds_val_binary, average='macro'),
    'roc_auc': roc_auc_score(y_val, ensemble_preds_val)
}

print(f"\nEnsemble - Validation F1: {ensemble_val_metrics['f1']:.4f}, ROC-AUC: {ensemble_val_metrics['roc_auc']:.4f}")

In [ ]:
# Compare all approaches
print("\nComparison of All Approaches:")
print("="*70)
print(f"{'Model':<35} {'Val F1':<12} {'Val ROC-AUC':<12}")
print("-"*70)
print(f"{'Original Best (L2 0.001)':<35} {l2_results[0]['metrics']['f1']:<12.4f} {l2_results[0]['metrics']['roc_auc']:<12.4f}")
print(f"{'Combined L2 + Dropout':<35} {combined_val_metrics['f1']:<12.4f} {combined_val_metrics['roc_auc']:<12.4f}")
print(f"{'Ensemble (3 models)':<35} {ensemble_val_metrics['f1']:<12.4f} {ensemble_val_metrics['roc_auc']:<12.4f}")
print("="*70)

# Select best approach
if ensemble_val_metrics['f1'] > max(l2_results[0]['metrics']['f1'], combined_val_metrics['f1']):
    print("\nBest approach: Ensemble")
    best_bonus_model = 'ensemble'
elif combined_val_metrics['f1'] > l2_results[0]['metrics']['f1']:
    print("\nBest approach: Combined L2 + Dropout")
    best_bonus_model = model_combined
else:
    print("\nBest approach: Original L2 model")
    best_bonus_model = l2_results[0]['model']

In [ ]:
# Evaluate best bonus model on test set
if best_bonus_model == 'ensemble':
    print("\nEvaluating ensemble on test set...")
    ensemble_preds_test = np.mean([model.predict(Xte, verbose=0) for model in ensemble_models], axis=0)
    ensemble_preds_test_binary = (ensemble_preds_test > 0.5).astype(int)
    
    test_precision = precision_score(y_test, ensemble_preds_test_binary, average='macro')
    test_recall = recall_score(y_test, ensemble_preds_test_binary, average='macro')
    test_f1 = f1_score(y_test, ensemble_preds_test_binary, average='macro')
    test_roc_auc = roc_auc_score(y_test, ensemble_preds_test)
    
    print(f"\nEnsemble Test Set Performance:")
    print(f"{'='*50}")
    print(f"Precision (macro): {test_precision:.4f}")
    print(f"Recall (macro):    {test_recall:.4f}")
    print(f"F1 Score (macro):  {test_f1:.4f}")
    print(f"ROC-AUC:           {test_roc_auc:.4f}")
    print(f"{'='*50}")
else:
    bonus_test_results = evaluate_model(best_bonus_model, Xte, y_test, "Bonus Model Test")

### Results and Analysis - Bonus

Tested two improvement strategies. Combined L2 and Dropout achieved validation F1 0.8879, worse than L2 alone at 0.9058. The ensemble of 3 models with different seeds achieved validation F1 0.9019, slightly below the original L2 model.

The original L2 model remains the best performer. Adding dropout to L2 regularization appears to over-regularize the model, reducing performance. The ensemble approach provides slight variance reduction but insufficient improvement to justify the computational cost.

Test performance matches the original best model at F1 0.8637, confirming that simpler L2 regularization alone is optimal for this dataset and architecture. Further improvements would likely require different architectural choices or feature engineering rather than additional regularization.